# 1. Loading necessary dependencies

- Import required packages.
- Load data into dataframe.

- Mount google drive to allow reading of data from drive.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

- Import dependencies

In [0]:
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import datetime
from google.colab import files

- Read file from file path in drive.
- Worksheet consists of 2 sheets read seperately into sheet 1 and sheet 2.

# 2. Objectives: Find vehicles that are not being used.
### Find vehicles that are not being used regularly.

---
#### 1. We have 3 dataframes (2 similar but covering different time period and one containing information regarding unique vehicle registrations), we combine the first 2 df and use it to compare the open leases df for registration numbers we need to work with.

#### We identify vehicles that have been returned.
---
#### 2. Visualisations

#### Plot of Distribution of Vehicle Activity over the one year period.
---
#### 3. Conclusion
#### Output CSV with Columns [Registration,	Start Date,	AvgDaysActiveInWeek,	DaysOfActivity,	WeeksActiveInYear,	LeaseDays,	EstimatedUtilisation, UtilisationPercentage]


# 3. Data Processing

## Import Excel files.

## 3.1. Load and Combine excel sheet to one Dataframe.

In [0]:
def load_and_combine_data(file_path1, file_path2):
  """
  function description:

  Loads data from the two Excel files, combine sheets from the first file,
  and return the combined dataframe and the dataframe from the second file.

  Args:
    file_path1: Path to the first Excel file (containing multiple sheets).
    file_path2: Path to the second Excel file (single sheet).

  Returns:
    A tuple containing:
      - combined_df: Dataframe combining sheets from the first file.
      - open_leases: Dataframe from the second file.

      We use tuple to protect the data from being modified.
      @return: combined_df, open_leases (use .copy() when assigning variable)
  """
  # Read sheets from the first file
  sheet1 = pd.read_excel(file_path1, 'September 23 to April 24')
  sheet2 = pd.read_excel(file_path1, 'May 24 to Aug 24')

  # Combine sheets into a single dataframe
  combined_df = pd.concat([sheet1, sheet2], ignore_index=True)

  # Read data from the second file
  open_leases = pd.read_excel(file_path2)

  return combined_df, open_leases


In [0]:
# specify path to file
file_path1 = '/content/drive/MyDrive/Colab Notebooks/TfL/Vehicle_Util/Navman_Telematics_Download_2023-09-01_to_2024-08-31_anonymised.xlsx'
file_path2 = '/content/drive/MyDrive/Colab Notebooks/TfL/Vehicle_Util/Open_Leases_2024-10-16_anonymised.xlsx'

# call function
combined_df, open_leases = load_and_combine_data(file_path1, file_path2)

In [0]:
def display_dataframe_info(df, df_name="Dataframe"):


    print(f"\n--- {df_name} ---")
    print("\nDataframe Info:")
    df.info()  # Display basic information

In [0]:
display_dataframe_info(combined_df, "Combined Dataframe")
display_dataframe_info(open_leases, "Open Leases Dataframe")

## 3.2. Preprocessing Combined Dataframe.

### 3.2.1. Preprocessing

Check how Vehicle and Registration columns difer from each other.

In [0]:
# Check if the number of unique values is indeed the same
num_unique_vehicles = combined_df['Vehicle'].nunique()
num_unique_registrations = combined_df['Registration'].nunique()

print(f"Number of unique vehicles: {num_unique_vehicles}")
print(f"Number of unique registrations: {num_unique_registrations}")

# Check if unique values are identical
unique_vehicles = combined_df['Vehicle'].unique()
unique_registrations = combined_df['Registration'].unique()

is_same_info = set(unique_vehicles) == set(unique_registrations)

print(f"Do 'Vehicle' and 'Registration' contain the same unique values? {is_same_info}")

different_rows = combined_df[combined_df['Registration'] != combined_df['Vehicle']]

print(f'The total number of rows where difference is noticed: {len(different_rows)}')

print('\n', different_rows[['Vehicle','Registration']].head())

In [0]:
def preprocess_vehicle_data(df):
    """
    Preprocesses the DataFrame by adding 'Returned' column, cleaning 'Registration',
    and formatting 'EventTime'.

    Args:
        df: The pandas DataFrame containing vehicle data.

    Returns:
        The preprocessed pandas DataFrame.
    """
    df['Returned'] = df['Vehicle'].str.contains(r'\(.*\)', regex=True).astype(int)
    df['Registration'] = df['Registration'].str.replace(' ', '')
    df['EventTime'] = pd.to_datetime(df['ActivityDate'].astype(str) + ' ' + df['EventTime'].astype(str))
    return df

In [0]:
combined_df_processed = preprocess_vehicle_data(combined_df.copy())  # Create a copy for preprocessing

### 3.2.2. Selecting the registration number we would like to work with

#### 3.2.2.1. DataFrame Summary.

In [0]:
open_leases['Reg No'].duplicated().sum() # check open_leases for duplicate Vehicles.

In [0]:
print(f'The number of Vehicles in open_leases df is: {open_leases["Reg No"].nunique()}')

In [0]:
print(f'The number of Vehicles in combined df is: {combined_df["Registration"].nunique()}')

#### 3.2.2.2. Filtering DataFrame

In [0]:
registration_to_keep = open_leases['Reg No'].unique().tolist() # specify registrations to keep by specifying registrations contained within open leases dataframe

In [0]:
filtered_combined_df = combined_df_processed[combined_df_processed['Registration'].isin(registration_to_keep)] # Filter combined df by registrations to keep

In [0]:
filtered_combined_df['Registration'].nunique()

In [0]:
# Registrations in open leases but not in telematics data

# Check registration in open_leases and check iof it exists in processed combined df
registration_not_in_filtered_combined_df = [reg for reg in open_leases['Reg No'].unique() if reg not in combined_df_processed['Registration'].unique()]
print(f'The number of Vehicles in open leases but not in telematics data is: {len(registration_not_in_filtered_combined_df)}')

registration_not_in_filtered_combined_df = pd.DataFrame({'Registration': registration_not_in_filtered_combined_df}) #make into df to export to csv

registration_not_in_filtered_combined_df.to_csv('registration_not_in_filtered_combined_df.csv', index=False)
# files.download('registration_not_in_filtered_combined_df.csv') # Remove (#) to download list of registrations and include back to stop code from downloading everytime cell is ran/ apply same below

# Registrations in telematics data but not in open leases

registration_not_in_open_leases = [reg for reg in combined_df_processed['Registration'].unique() if reg not in open_leases['Reg No'].values]
print(f'The number of Vehicles in telematics but not in open leases data is: {len(registration_not_in_open_leases)}')


registration_not_in_open_leases_df = pd.DataFrame({'Registration': registration_not_in_open_leases})

registration_not_in_open_leases_df.to_csv('registration_not_in_open_leases.csv', index=False)
# files.download('registration_not_in_open_leases.csv')

- There are 961 unique registration in the combined_df
- There are 866 unique registration in the open_leases data
- This leaves a difference of 95, where combined df has more unique registration
- checking the registrations that are in open leases but not in combined_df we have 81
- While checking the registration that are in combined_df but not in open leases we have 176 which is a total of 81 in open leases but not in combined_df and 95 in combined df but not in open leases.

In [0]:
# Drop rows where 'Returned' column is equal to 1
filtered_combined_df = filtered_combined_df[filtered_combined_df['Returned'] != 1]

- Written above we drop the rows where the vehicle has been returned as we do not want to include them in our analysis.
- Only one row was dropped as seen below cut from 785 to 784.

In [0]:
filtered_combined_df['Registration'].nunique()

# 4. Calculating activity days by Registration.

## 4.1. Overall Active Days.

In [0]:
def calculate_vehicle_activity_summary(df, registration_col='Registration', date_col='ActivityDate'):
    """
    Calculate the total activities and unique activity days for each vehicle.

    Args:
        df: The input pandas DataFrame containing vehicle activity data.
        registration_col: The name of the column containing vehicle registration information (default: 'Registration').
        date_col: The name of the column containing activity dates (default: 'ActivityDate').

    Returns:
        A pandas DataFrame summarizing total activities and unique activity days for each vehicle.
    """

    # Ensure date column is in datetime format
    df[date_col] = pd.to_datetime(df[date_col])

    # Calculate total activities
    total_activities = df.groupby(registration_col)[date_col].count().reset_index()
    total_activities = total_activities.rename(columns={date_col: 'TotalActivities'})

    # Calculate unique activity days
    activity_days = df.groupby(registration_col)[date_col].nunique().reset_index()
    activity_days = activity_days.rename(columns={date_col: 'DaysOfActivity'})

    # Merge results
    activity_summary = pd.merge(total_activities, activity_days, on=registration_col, validate = 'one_to_one')

    return activity_summary

In [0]:
activity_summary = calculate_vehicle_activity_summary(filtered_combined_df)

# Display activity_summary_df
print(activity_summary.sort_values(by='DaysOfActivity', ascending=False).head())

In [0]:
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
sns.histplot(activity_summary['DaysOfActivity'], bins=30, kde=False)  # Adjust bins as needed
plt.title('Distribution of Vehicle Activity Days')
plt.xlabel('Days of Activity')
plt.ylabel('Number of Vehicles')
plt.show()

## 4.2. Days Active In a Specific Week.

In [0]:
# Use iso standard for weekly dating for uniformity and reuseability, check start date of data and end date

date = datetime.date(2023, 9, 4) # 2024-08-30/2023-09-01 both start and end date fall within same iso week but for different years.
iso_calendar = date.isocalendar()

print(f"ISO Week Number: {iso_calendar[1]}")
print(f"ISO Year: {iso_calendar[0]}")

In [0]:
filtered_combined_df[filtered_combined_df['Registration'] == 'LS21DMX']

ISO week 2024 and 2023 overlap which leads to the weeks being read as one extra and also the overlap in the previous 8 day activity days.

In [0]:
# Extract ISO week number and year from 'ActivityDate'
filtered_combined_df['ISOWeek'] = filtered_combined_df['ActivityDate'].dt.isocalendar().week
filtered_combined_df['ISOYear'] = filtered_combined_df['ActivityDate'].dt.isocalendar().year

# Group by 'Registration', 'ISOWeek', 'ISOYear', and 'AcquisitionYear'
weekly_activity = filtered_combined_df.groupby(['Registration', 'ISOWeek', 'ISOYear'])['ActivityDate'].nunique().reset_index()

# Rename the column to 'DaysActiveInWeek'
weekly_activity = weekly_activity.rename(columns={'ActivityDate': 'DaysActiveInWeek'})

In [0]:
weekly_activity.head()

In [0]:
weekly_activity['DaysActiveInWeek'].unique()

In [0]:
# Create a histogram of 'DaysActiveInWeek'
plt.figure(figsize=(10, 6))
sns.histplot(weekly_activity['DaysActiveInWeek'], bins=7, discrete=True)
plt.title('Distribution of Days Active per Week for All Activities')
plt.xlabel('Days Active in a Week')
plt.ylabel('Number of Vehicles')
plt.show()

# 5. Estimating Underutilisation Rate.

## 5.1. Average Week Stats.

In [0]:
weekly_stats_by_reg = weekly_activity.groupby('Registration')['DaysActiveInWeek'].agg('mean').reset_index()
weekly_stats_by_reg = weekly_stats_by_reg.rename(columns={
    'DaysActiveInWeek': 'AvgDaysActiveInWeek'
})

In [0]:
# Identify and display null
print("Null Rows:\n", weekly_stats_by_reg[weekly_stats_by_reg.isnull().any(axis=1)])

In [0]:
# Round columns to the nearest whole number using standard rounding
weekly_stats_by_reg['AvgDaysActiveInWeek'] = weekly_stats_by_reg['AvgDaysActiveInWeek'].round().astype(int)

In [0]:
#Histogram of 'AvgDaysActiveInWeek'
plt.figure(figsize=(10, 6))
sns.histplot(weekly_stats_by_reg['AvgDaysActiveInWeek'], bins=7, discrete=True)
plt.title('Distribution of Days Active per Week for All Vehicles')
plt.xlabel('Average Days Active in a Week')
plt.ylabel('Number of Vehicles')
plt.show()

In [0]:
weekly_stats_by_reg['AvgDaysActiveInWeek'].value_counts().sort_index()

In [0]:
# Group by Activity Levels
weekly_stats_by_reg['ActivityLevel'] = pd.cut(weekly_stats_by_reg['AvgDaysActiveInWeek'],
                                               bins=[0, 2, 4, 7],
                                               labels=['Low', 'Medium', 'High'])
activity_level_counts = weekly_stats_by_reg['ActivityLevel'].value_counts().sort_index()

# Plot activity level counts
plt.figure(figsize=(8, 6))
sns.barplot(x=activity_level_counts.index, y=activity_level_counts.values)
plt.title('Distribution of Vehicle Activity Levels')
plt.xlabel('Activity Level')
plt.ylabel('Number of Vehicles')
plt.show()

In [0]:
weekly_activity.head()

In [0]:
# Find weeks active in Year
weekly_activity['IsActive'] = weekly_activity['DaysActiveInWeek'] > 0  # Create a boolean column indicating active weeks
weeks_active_by_reg = weekly_activity.groupby('Registration')['IsActive'].sum().reset_index()
weeks_active_by_reg = weeks_active_by_reg.rename(columns={'IsActive': 'WeeksActiveInYear'})

# Merge weeks active back into weekly_stats_by_reg
weekly_stats_by_reg = weekly_stats_by_reg.merge(weeks_active_by_reg, on='Registration')

In [0]:
weekly_stats_by_reg.head()

In [0]:
activity_summary.head()

In [0]:
stats_df = pd.merge(weekly_stats_by_reg,
                               activity_summary[['Registration', 'DaysOfActivity']],
                               on='Registration',
                               how='left')

In [0]:
stats_df.head()

## 5.2. Estimate Utilisation Rate.

In [0]:
merged_df = pd.merge(stats_df, open_leases[['Reg No', 'Start Date']], left_on='Registration', right_on='Reg No', how='left', validate='one_to_one')
merged_df.drop(columns=['Reg No'], inplace=True)

In [0]:
new_order = ['Registration', 'Start Date', 'AvgDaysActiveInWeek', 'WeeksActiveInYear', 'DaysOfActivity']
merged_df = merged_df[new_order]

In [0]:
merged_df.head()

In [0]:
filtered_combined_df['ActivityDate'].max() # We confirm the max date using the filtered df as it contains all preprocessed and filtered vehicles we are considering

In [0]:
# find the max value of activity date to determine the last date we are working with
max_activity_date = filtered_combined_df['ActivityDate'].max()

# Calculate the cutoff date, which is one year before the max activity date
cutoff_date = max_activity_date - pd.DateOffset(years=1)

# Create a new column called 'LeaseDays' and initialize it with 0
merged_df['LeaseDays'] = 0

# Iterate through rows (Start Date) and calculate days difference
for index, row in merged_df.iterrows():
    if row['Start Date'] <= cutoff_date:
        merged_df.at[index, 'LeaseDays'] = 365  # Assign 365 if start date is one year or more before the max date
    else:
        merged_df.at[index, 'LeaseDays'] = (max_activity_date - row['Start Date']).days


In [0]:
cutoff_date

In [0]:
merged_df.isnull().sum()

In [0]:
# Calculate Utilisation Rate
merged_df['UtilisationRate'] = merged_df['DaysOfActivity'] / merged_df['LeaseDays']

# Calculate Utilisation Percentage
merged_df['UtilisationPercentage'] = merged_df['UtilisationRate'] * 100
merged_df['UtilisationPercentage'] = merged_df['UtilisationPercentage'].round(2)

In [0]:
merged_df.sort_values(by='UtilisationPercentage', ascending=True).head(20)

- Plor Distribution of Utilisation Percentage helps us with setting cut off for utilisation rate and see how amny vehicles might be affected.

In [0]:
plt.figure(figsize=(10, 6))
ax = sns.histplot(x=merged_df['UtilisationPercentage'], bins=5)
plt.title('Distribution of Utilisation Percentage')
plt.xlabel('Percentage')
plt.ylabel('Frequency')
plt.show()

In [0]:
merged_df['UtilisationPercentage'].median()

In [0]:
}merged_df['UtilisationPercentage'].max()

# 6. Export Result.

In [0]:
result_df = merged_df[['Registration', 'Start Date', 'AvgDaysActiveInWeek', 'DaysOfActivity', 'LeaseDays', 'UtilisationPercentage']].copy()

In [0]:
result_df = result_df.sort_values(by='UtilisationPercentage', ascending=True)

# Export to CSV
result_df.to_csv('result_df.csv', index=False)
files.download('result_df.csv')

# 7. Bonus: Confirm utilisation using Trip Distance.

In [0]:
combined_df[combined_df['Registration'] == 'MW69XCJ']

In [0]:
open_leases[open_leases['Reg No'] == 'MW69XCJ']

In [0]:
total_trip_distance = filtered_combined_df.groupby(['Registration', 'ActivityDate'])['TripDistance'].sum().reset_index()

In [0]:
total_trip_distance[total_trip_distance['Registration'] == 'MW69XCJ']['TripDistance'].sum()

In [0]:
total_trip_distance[total_trip_distance['Registration'] == 'MW69XCJ']